In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from astropy.utils.data import get_pkg_data_filename
import subprocess as sp
import glob
import astropy.io.fits as pyfits


# This notebook:
<ol>
<li>Formats raw data for Science</li>
<li>Creates one image per filter</li>
<li>Sets up files to get PSF</li>
</ol>

In [2]:
#a fcn to create a file to use for PARAMETERS_NAME input to use for SExtractor (Bruno's code)


def scampparams():
    #list of parameters for SExtractor we want
    pars = ["XWIN_IMAGE", "YWIN_IMAGE", "ERRAWIN_IMAGE", "ERRBWIN_IMAGE",\
            "ERRTHETAWIN_IMAGE", "FLUX_AUTO", "FLUXERR_AUTO", "FLAGS",\
            "FLAGS_WEIGHT", "FLUX_RADIUS", "MAG_AUTO","ALPHA_J2000","DELTA_J2000"]
    f = open("scamp.param","w")
    for p in pars:
        f.write(f"{p}\n")
    f.close()
    return None

In [3]:
#excutes the fcn above to ACTUALLY generate the file
scampparams()

In [4]:
#defining needed things specific to this project

#need a dictionary for zeropoint values of each filter
zeropoints = {"F435W":25.665,"F606W":26.493,"F814W":25.97,"F105W":26.2687,"F125W":26.2303,"F140W":24.4524,\
              "F160W":25.9463, "F625W":24.85714771, "F775W":25.66195995, "F850LP":25.89943399}

#need to create a dictionary for the 10 indiviual filters
filters_used = {'F435W','F606W','F814W','F625W','F775W','F850LP','F105W','F125W','F140W','F160W'}

filterImages = {}
all_filter_files = glob.glob('/data1/rowland/all_fits_data/hst*.fits')


# 1. Splitting 'sci' & 'wht' files

In [5]:
# need to split 3 files from hst fits data files for sextractor intake reasons


for image in all_filter_files:
        #print('this is image',image)
        hdu = pyfits.open(image)
        header = hdu[0].header
        f = ""
        f1 = ""
        f2 = ""
        try:
            f = header["FILTER"] 
        except KeyError:
                pass
        try: 
             f1 = header["FILTER1"]
        except KeyError: 
                    pass
        try:
            f2 = header["FILTER2"]
        except KeyError:
                pass
            
        if f in filters_used:
            filterName = f
        elif f1 in filters_used:
            filterName = f1
        elif f2 in filters_used:
            filterName = f2 
        else:
            print('error determining filter used - likely a total file with not one filter, but check')
            continue
        mygain =  float(hdu[0].header["CCDGAIN"])
        #print('my test of gain values:',mygain)
        gain = 2 #float(hdu[0].header["CCDGAIN"])
        exptime = float(hdu[0].header["EXPTIME"])

        if filterName not in filterImages.keys():
            filterImages[filterName] = []

        #print("%s_sci.fits"%(image[:-5])) #thats the output below of sci file locations
        headerOut = hdu["SCI"].header
        headerOut["FILTER"] = filterName
        headerOut["RADESYS"] = 'ICRS'
        headerOut["EXPTIME"] = exptime
        headerOut["GAIN"] = gain ###COULD change gain here
        name = "/data1/rowland/all_fits_data/sci_n_wht_files/"
        outname = name + image.split("/")[-1][:-5]+"_sci.fits" 
        dataOUt = hdu["SCI"].data/gain #* hdu['EXP'].data ####COULD change gain here
        #print('this is outname:',outname)
        pyfits.writeto(outname,dataOUt,header=headerOut,overwrite=True) 
        pyfits.writeto(name + image.split("/")[-1][:-5]+"_sci.wht.fits",hdu["WHT"].data,header=hdu["WHT"].header,overwrite=True)
        filterImages[filterName].append(outname)
        hdu.close() #**think** this will still work for writing but saves memory
        

print('FILTER IMAGES printout:',filterImages)

error determining filter used - likely a total file with not one filter, but check
error determining filter used - likely a total file with not one filter, but check
error determining filter used - likely a total file with not one filter, but check
error determining filter used - likely a total file with not one filter, but check
error determining filter used - likely a total file with not one filter, but check
error determining filter used - likely a total file with not one filter, but check
FILTER IMAGES printout: {'F160W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci.fits', '/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci.fits'], 'F105W': ['/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci.fits', '/data1

# 2. SExtractor start to create One Image per filter (i.e. elgordo_{filter}.fits)

In [6]:
# 2nd part of Bruno's code that actually runs sextractor AMENDED by me to run in jupyter nb


thresh = 50.0
skycat = "SDSS-R9"  #"2MASS"#"USNO-B1"#

for filter in filterImages.keys(): 
   # if not filter in ['F125W','F814W']:
   #     continue
    catalogs = ""
    for image in filterImages[filter]:
        root = image[:-5]
        catalogs += f" {root}.cat"
        print(root)
        ! sex  {image} -DETECT_THRESH {thresh} -CATALOG_TYPE FITS_LDAC -CATALOG_NAME {root}.cat -CHECKIMAGE_TYPE NONE -PARAMETERS_NAME scamp.param -WEIGHT_TYPE NONE -FILTER Y -FILTER_NAME gauss_1.5_3x3.conv -SEEING_FWHM 0.5 -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh} -DETECT_THRESH {thresh}

    all_images = " ".join(filterImages[filter])
   
    ! swarp -IMAGEOUT_NAME /data1/rowland/elgordo_{filter}.fits -COMBINE_TYPE SUM -PIXELSCALE_TYPE MANUAL -PIXEL_SCALE 0.06 -GAIN_KEYWORD GAIN -WEIGHT_TYPE MAP_WEIGHT -WEIGHT_SUFFIX .wht.fits {all_images}


/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f160w_drz_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f160w_drz_SE_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f160w_drz_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f160w_drz_SE_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f105w_drz_SE_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f105w_drz_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f105w_drz_SE_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f105w_drz_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i2_wfc3_ir_f140w_drz_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j2_wfc3_ir_f140w_drz_SE_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_i4_wfc3_ir_f140w_drz_sci
/data1/rowland/all_fits_data/sci_n_wht_files/hst_14096_j4_wfc3_ir_f140w_drz_SE_sci
/data1/rowland/all_fit

In [7]:
# FOR GALFIT must UPDATE EXPTIME =1

all_elgordo = glob.glob('/data1/rowland/elgordo*.fits')

for image in all_elgordo:
    hdu = pyfits.open(image)
    header = hdu[0].header     
    dataout = hdu[0].data
    header["EXPTIME"] = 1
    pyfits.writeto(image,dataout, header=header,overwrite=True)
    hdu.close() 

# 3. SExtractor start to create PSFs per filter (ran on elgordo_{filter}.fits)

In [8]:
elgordo_files = {'F606W': '/data1/rowland/elgordo_F606W.fits', \
              'F625W': '/data1/rowland/elgordo_F625W.fits', \
              'F775W': '/data1/rowland/elgordo_F775W.fits', \
              'F814W': '/data1/rowland/elgordo_F814W.fits', \
              'F850LP': '/data1/rowland/elgordo_F850LP.fits',\
              'F105W': '/data1/rowland/elgordo_F105W.fits',\
              'F125W': '/data1/rowland/elgordo_F125W.fits',\
              'F140W': '/data1/rowland/elgordo_F140W.fits',\
              'F160W': '/data1/rowland/elgordo_F160W.fits',
              'F435W': '/data1/rowland/elgordo_F435W.fits'}


thresh_test = 5.0 #3.0

for filter,image in elgordo_files.items():
    #if not filter == 'F125W':
    #    continue
    print('processing this image',image)
    ! sex {image} -DETECT_THRESH {thresh_test} -CATALOG_NAME /data1/rowland/testPSF_{filter}.cat -PARAMETERS_NAME default.param -PIXEL_SCALE 0.12 -MAG_ZEROPOINT {zeropoints[filter]} -ANALYSIS_THRESH {thresh_test} -DETECT_THRESH {thresh_test} -CHECKIMAGE_NAME  /data1/rowland/segmaps_check/{filter}_check.fits -CHECKIMAGE_TYPE SEGMENTATION


processing this image /data1/rowland/elgordo_F606W.fits
> 
----- SExtractor 2.19.5 started on 2019-07-02 at 16:08:09 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing check-image(s)
> Initializing catalog
> Looking for elgordo_F606W.fits
----- Measuring from: elgordo_F606W.fits
      "Unnamed" / no ext. header / 8201x8473 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  

> Line:  425  Objects:        1 detected /        0 sextracted
> Line:  450  Objects:        1 detected /        0 sextracted
> Line:  475  Objects:        3 detected /        0 sextracted
> Line:  500  Objects:        4 detected /        0 sextracted
> Line:  525  Objects:        9 detected /        0 sextracted
> Line:  550  Objects:       19 detected /        0 sextracted
> Line:  575  Objects:       19 detected /        0 sextracted
> Line:  600  Objects:       20 detected /        0 sextracted
> Line:  625  Objects:       26 detected /        0 sextracted
> Line:  650  Objects:       30 detected /        0 sextracted
> Line:  675  Objects:       31 detected /        0 sextracted
> Line:  700  Objects:       34 detected /        0 sextracted
> Line:  725  Objects:       41 detected /        0 sextracted
> Line:  750  Objects:       48 detected /        0 sextracted
> Line:  775  Objects:       59 detected /        0 sextracted
> Line:  800  Objects:       63 detected /        0 sex

> Line: 3275  Objects:     1818 detected /      892 sextracted
> Line: 3300  Objects:     1838 detected /      914 sextracted
> Line: 3325  Objects:     1863 detected /      926 sextracted
> Line: 3350  Objects:     1878 detected /      947 sextracted
> Line: 3375  Objects:     1892 detected /      971 sextracted
> Line: 3400  Objects:     1921 detected /      987 sextracted
> Line: 3425  Objects:     1941 detected /      997 sextracted
> Line: 3450  Objects:     1958 detected /     1020 sextracted
> Line: 3475  Objects:     1974 detected /     1034 sextracted
> Line: 3500  Objects:     2001 detected /     1058 sextracted
> Line: 3525  Objects:     2018 detected /     1082 sextracted
> Line: 3550  Objects:     2037 detected /     1103 sextracted
> Line: 3575  Objects:     2069 detected /     1119 sextracted
> Line: 3600  Objects:     2089 detected /     1133 sextracted
> Line: 3625  Objects:     2108 detected /     1158 sextracted
> Line: 3650  Objects:     2124 detected /     1181 sex

> Line: 6050  Objects:     4149 detected /     3023 sextracted
> Line: 6075  Objects:     4166 detected /     3036 sextracted
> Line: 6100  Objects:     4190 detected /     3048 sextracted
> Line: 6125  Objects:     4205 detected /     3059 sextracted
> Line: 6150  Objects:     4224 detected /     3080 sextracted
> Line: 6175  Objects:     4243 detected /     3101 sextracted
> Line: 6200  Objects:     4267 detected /     3123 sextracted
> Line: 6225  Objects:     4296 detected /     3147 sextracted
> Line: 6250  Objects:     4309 detected /     3161 sextracted
> Line: 6275  Objects:     4335 detected /     3177 sextracted
> Line: 6300  Objects:     4361 detected /     3192 sextracted
> Line: 6318  Objects:     4380 detected /     3200 sextracted
> Line: 6325  Objects:     4389 detected /     3203 sextracted
> Line: 6350  Objects:     4408 detected /     3230 sextracted
> Line: 6375  Objects:     4426 detected /     3256 sextracted
> Line: 6400  Objects:     4450 detected /     3283 sex

> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background map at line:  832
> Setting up background map at line:  896
> Setting up background map at line:  960
> Setting up background map at line: 1024
> Setting up background map at line: 1088
> Setting up background map at line: 1152
> Setting up background map at line: 1216
> Setting up background map at line: 1280
> Setting up background map at line: 1344
> Setting up background map at line: 1408
> Setting up background map at line: 1472
> Setting up background map at line: 1536
> Setting up background map at line: 1600
> Setting up background map at line: 1664
> Setting up background map at line: 1728
> Setting up background map at line: 1792
> Setting up background map at line: 1856
> Setting up background map at line: 1920
> Setting up background map at lin

...

> Line: 2950  Objects:     1263 detected /      518 sextracted
> Line: 2975  Objects:     1287 detected /      528 sextracted
> Line: 3000  Objects:     1309 detected /      547 sextracted
> Line: 3025  Objects:     1328 detected /      563 sextracted
> Line: 3050  Objects:     1340 detected /      576 sextracted
> Line: 3075  Objects:     1351 detected /      591 sextracted
> Line: 3100  Objects:     1374 detected /      604 sextracted
> Line: 3125  Objects:     1394 detected /      618 sextracted
> Line: 3150  Objects:     1413 detected /      637 sextracted
> Line: 3175  Objects:     1436 detected /      652 sextracted
> Line: 3200  Objects:     1446 detected /      667 sextracted
> Line: 3225  Objects:     1459 detected /      680 sextracted
> Line: 3250  Objects:     1478 detected /      693 sextracted
> Line: 3275  Objects:     1482 detected /      708 sextracted
> Line: 3300  Objects:     1500 detected /      724 sextracted
> Line: 3325  Objects:     1514 detected /      738 sex

> Setting up background map at line: 2560
> Setting up background map at line: 2624
> Setting up background map at line: 2688
> Setting up background map at line: 2752
> Setting up background map at line: 2816
> Setting up background map at line: 2880
> Setting up background map at line: 2944
> Setting up background map at line: 3008
> Setting up background map at line: 3072
> Setting up background map at line: 3136
> Setting up background map at line: 3200
> Setting up background map at line: 3264
> Setting up background map at line: 3328
> Setting up background map at line: 3392
> Setting up background map at line: 3456
> Setting up background map at line: 3520
> Setting up background map at line: 3584
> Setting up background map at line: 3648
> Setting up background map at line: 3712
> Setting up background map at line: 3776
> Setting up background map at line: 3840
> Setting up background map at line: 3904
> Setting up background map at line: 3968
> Setting up background map at lin

> Line: 2200  Objects:     1347 detected /      351 sextracted
> Line: 2225  Objects:     1371 detected /      371 sextracted
> Line: 2250  Objects:     1396 detected /      384 sextracted
> Line: 2275  Objects:     1427 detected /      393 sextracted
> Line: 2288  Objects:     1438 detected /      400 sextracted
> Line: 2300  Objects:     1457 detected /      406 sextracted
> Line: 2325  Objects:     1479 detected /      425 sextracted
> Line: 2350  Objects:     1507 detected /      444 sextracted
> Line: 2375  Objects:     1532 detected /      465 sextracted
> Line: 2400  Objects:     1555 detected /      475 sextracted
> Line: 2425  Objects:     1577 detected /      491 sextracted
> Line: 2450  Objects:     1607 detected /      505 sextracted
> Line: 2475  Objects:     1626 detected /      530 sextracted
> Line: 2500  Objects:     1658 detected /      554 sextracted
> Line: 2525  Objects:     1674 detected /      577 sextracted
> Line: 2550  Objects:     1711 detected /      589 sex

> Initializing check-image(s)
> Initializing catalog
> Looking for elgordo_F435W.fits
----- Measuring from: elgordo_F435W.fits
      "Unnamed" / no ext. header / 4397x4256 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:   64
> Setting up background map at line:  128
> Setting up background map at line:  192
> Setting up background map at line:  256
> Setting up background map at line:  320
> Setting up background map at line:  384
> Setting up background map at line:  448
> Setting up background map at line:  512
> Setting up background map at line:  576
> Setting up background map at line:  640
> Setting up background map at line:  704
> Setting up background map at line:  768
> Setting up background map at line:  832
> Setting up background map at line:  896
> Setting up background map at line:  960
> Setting up background map at line: 1024
> Setting up background map at line: 1088
> Setting up background map at line: 

> Line: 1550  Objects:      339 detected /       38 sextracted
> Line: 1575  Objects:      346 detected /       44 sextracted
> Line: 1600  Objects:      353 detected /       50 sextracted
> Line: 1625  Objects:      359 detected /       56 sextracted
> Line: 1650  Objects:      372 detected /       62 sextracted
> Line: 1675  Objects:      378 detected /       66 sextracted
> Line: 1700  Objects:      391 detected /       73 sextracted
> Line: 1725  Objects:      402 detected /       83 sextracted
> Line: 1750  Objects:      406 detected /       93 sextracted
> Line: 1775  Objects:      417 detected /      106 sextracted
> Line: 1800  Objects:      432 detected /      108 sextracted
> Line: 1825  Objects:      445 detected /      117 sextracted
> Line: 1850  Objects:      453 detected /      124 sextracted
> Line: 1875  Objects:      461 detected /      128 sextracted
> Line: 1900  Objects:      466 detected /      139 sextracted
> Line: 1925  Objects:      478 detected /      147 sex

# Run 'get_psfs.py' in terminal to get PSF's per filter
### THEN
## Proceed to StepTwo notebook